# Sample Workflow for d3rlpy Experiments

In [73]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import itertools
import math
import subprocess
import os
import d3rlpy
plt.style.use('matplotlibrc')

from Python.data_sampler import *

## Building an MDPDataset

We first read in a large batch of samples from the file. As `d3rlpy` wants it in the form (observations, actions, rewards, terminal flags), we go ahead and do that. Here's a helper function to get a dataset from a list of chunks of your choosing.

In [163]:
def get_dataset(chunks : list, batch_size=30000, 
                path="collected_data/rl_det_small.txt") -> d3rlpy.dataset.MDPDataset :
    random.seed(0)
    samples = DataSampler(path_to_data=path)
    states = []
    actions = []
    rewards = []
    next_states = []
    for chunk in chunks:
        samples.use_chunk(chunk)
        samples.read_chunk()
        [statesChunk, actionsChunk, rewardsChunk, nextStatesChunk] = samples.get_batch(batch_size)
        states.append(statesChunk)
        actions.append(actionsChunk)
        rewards.append(rewardsChunk)
        next_states.append(nextStatesChunk)
    states = torch.cat(states)
    actions = torch.cat(actions)
    rewards = torch.cat(rewards)
    next_states = torch.cat(next_states)
    terminals = np.zeros(len(states))
    terminals[::100] = 1 #episode length 100, change if necessary
    print(states.shape)
    dataset = d3rlpy.dataset.MDPDataset(states.numpy(), 
                                        actions.numpy(), 
                                        rewards.numpy(), terminals)
    return dataset

We can build the dataset from there, just like this, and split into train and test sets.

In [164]:
dataset = get_dataset([3,5,7,9], path="collected_data/rl_det_small.txt")

[ 0.00000000e+00  7.95731469e+08 -8.17891077e-02 -1.19999531e-03
  7.39998658e-03  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
  2.09713430e-01 -2.63658359e-01  6.00000000e-01]
Read chunk # 4 out of 10000
[ 0.00000000e+00  7.95731469e+08  1.24610892e-01  2.40000469e-03
 -7.60001342e-03  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
 -2.20016566e-01  3.79282423e-01 -6.00000000e-01]
Read chunk # 6 out of 10000
[ 0.00000000e+00  7.95731469e+08 -9.01891077e-02  1.08000047e-02
  3.99986580e-04  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
 -3.17973088e-02 -2.40776052e-01  6.00000000e-01]
Read chunk # 8 out of 10000
[ 0.00000000e+00  7.95731469e+08  6.91108923e-02 -5.99999531e-03
 -6.00001342e-03  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
 -1.42355434e-01  2.22081792e-01 -6.00000000e-01]
Read chunk # 10 out of 10000
torch.Size([111080, 6])


In [165]:
print("The behavior policy value statistics are:")
dataset.compute_stats()['return']

The behavior policy value statistics are:


{'mean': -4.1227446,
 'std': 2.4676569,
 'min': -12.578855,
 'max': 0.0,
 'histogram': (array([ 26,   9,   7,   7,   8,   7,  10,  13,  27,  54,  56,  73, 109,
          84, 186, 148, 124,  83,  67,  13]),
  array([-12.578855 , -11.949912 , -11.320969 , -10.692026 , -10.063084 ,
          -9.434141 ,  -8.805199 ,  -8.176255 ,  -7.5473127,  -6.9183702,
          -6.2894273,  -5.6604843,  -5.031542 ,  -4.4025993,  -3.7736564,
          -3.1447136,  -2.515771 ,  -1.8868282,  -1.2578855,  -0.6289427,
           0.       ], dtype=float32))}

In [166]:
from sklearn.model_selection import train_test_split
train_episodes, test_episodes = train_test_split(dataset, test_size=0.2)

## Setting up an Algorithm

In [167]:
from d3rlpy.algos import CQL
model = CQL(q_func_factory='mean', #qr -> quantile regression q function, but you don't have to use this
            reward_scaler='standard',
          actor_learning_rate=1e-5, 
          critic_learning_rate=0.0003, 
            use_gpu=False) #change it to true if you have one
model.build_with_dataset(dataset)

In [168]:
from d3rlpy.metrics.scorer import td_error_scorer
from d3rlpy.metrics.scorer import average_value_estimation_scorer
from d3rlpy.metrics.scorer import initial_state_value_estimation_scorer

# calculate metrics with test dataset
ave_error_init = average_value_estimation_scorer(model, test_episodes)
print(ave_error_init)

0.005242250281842394


In [150]:
%load_ext tensorboard
%tensorboard --logdir runs

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


Reusing TensorBoard on port 6006 (pid 77774), started 0:01:16 ago. (Use '!kill 77774' to kill it.)

In [169]:
model.fit(train_episodes,
        eval_episodes=test_episodes,
        n_epochs=5, 
        tensorboard_dir='runs',
        scorers={
            'td_error': td_error_scorer,
            'init_value': initial_state_value_estimation_scorer,
            'ave_value': average_value_estimation_scorer
        })

2022-04-06 21:00.09 [debug    ] RoundIterator is selected.
2022-04-06 21:00.09 [info     ] Directory is created at d3rlpy_logs/CQL_20220406210009
2022-04-06 21:00.09 [debug    ] Fitting reward scaler...       reward_scaler=standard
2022-04-06 21:00.09 [warning  ] Skip building models since they're already built.
2022-04-06 21:00.09 [info     ] Parameters are saved to d3rlpy_logs/CQL_20220406210009/params.json params={'action_scaler': None, 'actor_encoder_factory': {'type': 'default', 'params': {'activation': 'relu', 'use_batch_norm': False, 'dropout_rate': None}}, 'actor_learning_rate': 1e-05, 'actor_optim_factory': {'optim_cls': 'Adam', 'betas': (0.9, 0.999), 'eps': 1e-08, 'weight_decay': 0, 'amsgrad': False}, 'alpha_learning_rate': 0.0001, 'alpha_optim_factory': {'optim_cls': 'Adam', 'betas': (0.9, 0.999), 'eps': 1e-08, 'weight_decay': 0, 'amsgrad': False}, 'alpha_threshold': 10.0, 'batch_size': 256, 'conservative_weight': 5.0, 'critic_encoder_factory': {'type': 'default', 'params': 

Epoch 1/5:   0%|          | 0/343 [00:00<?, ?it/s]

2022-04-06 21:00.33 [info     ] CQL_20220406210009: epoch=1 step=343 epoch=1 metrics={'time_sample_batch': 0.0003157273673455152, 'time_algorithm_update': 0.06868734651682329, 'temp_loss': 4.5058160120127155, 'temp': 0.9832170249421812, 'alpha_loss': -7.1547618615807576, 'alpha': 1.0126195798462403, 'critic_loss': 8.009437349725395, 'actor_loss': 0.6563751490965877, 'time_step': 0.06907681195450942, 'td_error': 0.998184895395473, 'init_value': 0.32236340641975403, 'ave_value': 0.31981140022080984} step=343
2022-04-06 21:00.33 [info     ] Model parameters are saved to d3rlpy_logs/CQL_20220406210009/model_343.pt


Epoch 2/5:   0%|          | 0/343 [00:00<?, ?it/s]

2022-04-06 21:00.58 [info     ] CQL_20220406210009: epoch=2 step=686 epoch=2 metrics={'time_sample_batch': 0.0003133056462679938, 'time_algorithm_update': 0.06994466392361388, 'temp_loss': 1.9120248182521964, 'temp': 0.9586843066243319, 'alpha_loss': 7.930571129301199, 'alpha': 1.0036906923218998, 'critic_loss': -7.2960928539492995, 'actor_loss': 2.731188000464926, 'time_step': 0.07033466389158377, 'td_error': 1.5712441166613447, 'init_value': 0.6769112944602966, 'ave_value': 0.6663637833781404} step=686
2022-04-06 21:00.58 [info     ] Model parameters are saved to d3rlpy_logs/CQL_20220406210009/model_686.pt


...

Epoch 4/5:   0%|          | 0/343 [00:00<?, ?it/s]

2022-04-06 21:01.52 [info     ] CQL_20220406210009: epoch=4 step=1372 epoch=4 metrics={'time_sample_batch': 0.00032186994747239716, 'time_algorithm_update': 0.07587018096412236, 'temp_loss': -0.3027828116290293, 'temp': 0.9479728283757024, 'alpha_loss': 16.860179915025004, 'alpha': 0.9206059844431307, 'critic_loss': -15.363678295480266, 'actor_loss': 4.954739303004985, 'time_step': 0.07627053163489517, 'td_error': 4.112356028059688, 'init_value': -1.096514344215393, 'ave_value': -1.1222196783244935} step=1372
2022-04-06 21:01.52 [info     ] Model parameters are saved to d3rlpy_logs/CQL_20220406210009/model_1372.pt


Epoch 5/5:   0%|          | 0/343 [00:00<?, ?it/s]

2022-04-06 21:02.18 [info     ] CQL_20220406210009: epoch=5 step=1715 epoch=5 metrics={'time_sample_batch': 0.00032411580878166, 'time_algorithm_update': 0.0754263831992191, 'temp_loss': -0.8014757160542658, 'temp': 0.9576302907557251, 'alpha_loss': 18.4448027541269, 'alpha': 0.8810043835431425, 'critic_loss': -15.968879897114835, 'actor_loss': 6.788558832062924, 'time_step': 0.0758308836044445, 'td_error': 5.324599808256394, 'init_value': -2.0982847213745117, 'ave_value': -2.1331112435323885} step=1715
2022-04-06 21:02.18 [info     ] Model parameters are saved to d3rlpy_logs/CQL_20220406210009/model_1715.pt


[(1,
  {'time_sample_batch': 0.0003157273673455152,
   'time_algorithm_update': 0.06868734651682329,
   'temp_loss': 4.5058160120127155,
   'temp': 0.9832170249421812,
   'alpha_loss': -7.1547618615807576,
   'alpha': 1.0126195798462403,
   'critic_loss': 8.009437349725395,
   'actor_loss': 0.6563751490965877,
   'time_step': 0.06907681195450942,
   'td_error': 0.998184895395473,
   'init_value': 0.32236340641975403,
   'ave_value': 0.31981140022080984}),
 (2,
  {'time_sample_batch': 0.0003133056462679938,
   'time_algorithm_update': 0.06994466392361388,
   'temp_loss': 1.9120248182521964,
   'temp': 0.9586843066243319,
   'alpha_loss': 7.930571129301199,
   'alpha': 1.0036906923218998,
   'critic_loss': -7.2960928539492995,
   'actor_loss': 2.731188000464926,
   'time_step': 0.07033466389158377,
   'td_error': 1.5712441166613447,
   'init_value': 0.6769112944602966,
   'ave_value': 0.6663637833781404}),
 (3,
  {'time_sample_batch': 0.0003292922028299671,
   'time_algorithm_update': 0.

## Off-Policy Evaluation

We do get some metrics on a test set of initial state value and average value. However, these estimates (using the critic's Q-function) of model performance are biased. They're useful for validation during training, but not much else. Instead, we fit a Q-function to the data (or a separate dataset, as I've done here) separately and evaluate the model's performance on it.

Feel free to change the chunks and number of steps.

In [171]:
from d3rlpy.ope import FQE
# metrics to evaluate with
from d3rlpy.metrics.scorer import soft_opc_scorer


ope_dataset = get_dataset([2,4,6,8], path="collected_data/rl_det_small.txt") #change if you'd prefer different chunks
ope_train_episodes, ope_test_episodes = train_test_split(ope_dataset, test_size=0.2)

fqe = FQE(algo=model, use_gpu=False) #change this if you have one!
fqe.fit(ope_train_episodes, eval_episodes=ope_test_episodes,
        tensorboard_dir='runs',
        n_epochs=50, n_steps_per_epoch=1000, #change if overfitting/underfitting
        scorers={
           'init_value': initial_state_value_estimation_scorer,
            'ave_value': average_value_estimation_scorer,
           'soft_opc': soft_opc_scorer(return_threshold=0)
        })

[ 0.00000000e+00  7.95731469e+08 -1.03891077e-02 -1.41999953e-02
 -2.10001342e-03  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
 -1.78778459e-03 -1.34615461e-02  4.84073546e-02]
Read chunk # 3 out of 10000
[ 0.00000000e+00  7.95731469e+08 -7.24891077e-02 -1.35999953e-02
 -4.20001342e-03  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
 -6.23311010e-02 -1.64283998e-01  6.00000000e-01]
Read chunk # 5 out of 10000
[ 0.00000000e+00  7.95731469e+08  7.01089229e-03 -4.19999531e-03
  7.39998658e-03  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
  2.21623335e-01 -2.86362315e-02 -8.00043364e-02]
Read chunk # 7 out of 10000
[ 0.00000000e+00  7.95731469e+08 -1.03989108e-01 -1.37999953e-02
  7.99998658e-03  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
  2.76352555e-01 -3.26280816e-01  6.00000000e-01]
Read chunk # 9 out of 10000
torch.Size([111080, 6])
2022-04-06 21:04.24 [debug    ] RoundIterator is selected.
2022-04-06 21:04.24 [info     ] Directory is created at d3rlpy_logs/FQE_2022040621042

Epoch 1/50:   0%|          | 0/878 [00:00<?, ?it/s]

2022-04-06 21:04.26 [info     ] FQE_20220406210424: epoch=1 step=878 epoch=1 metrics={'time_sample_batch': 0.00013119403213466218, 'time_algorithm_update': 0.0017099345192007702, 'loss': 0.0007221336903229773, 'time_step': 0.001900675084976509, 'init_value': -0.3511500060558319, 'ave_value': -0.3517933719735239, 'soft_opc': nan} step=878


2022-04-06 21:04.26 [info     ] Model parameters are saved to d3rlpy_logs/FQE_20220406210424/model_878.pt


Epoch 2/50:   0%|          | 0/878 [00:00<?, ?it/s]

...

2022-04-06 21:05.56 [info     ] Model parameters are saved to d3rlpy_logs/FQE_20220406210424/model_43022.pt


Epoch 50/50:   0%|          | 0/878 [00:00<?, ?it/s]

2022-04-06 21:05.58 [info     ] FQE_20220406210424: epoch=50 step=43900 epoch=50 metrics={'time_sample_batch': 0.00012076579899885659, 'time_algorithm_update': 0.00155189857396015, 'loss': 0.029719315855661427, 'time_step': 0.0017268255794238392, 'init_value': -1.707073450088501, 'ave_value': -1.7068879730584705, 'soft_opc': nan} step=43900


2022-04-06 21:05.58 [info     ] Model parameters are saved to d3rlpy_logs/FQE_20220406210424/model_43900.pt


[(1,
  {'time_sample_batch': 0.00013119403213466218,
   'time_algorithm_update': 0.0017099345192007702,
   'loss': 0.0007221336903229773,
   'time_step': 0.001900675084976509,
   'init_value': -0.3511500060558319,
   'ave_value': -0.3517933719735239,
   'soft_opc': nan}),
 (2,
  {'time_sample_batch': 0.00013680067040654142,
   'time_algorithm_update': 0.0018210438226513004,
   'loss': 0.0021344184700107043,
   'time_step': 0.0020162573164850812,
   'init_value': -0.571056067943573,
   'ave_value': -0.5718733890903176,
   'soft_opc': nan}),
 (3,
  {'time_sample_batch': 0.00013056404226733233,
   'time_algorithm_update': 0.0017468429643635326,
   'loss': 0.004262366834274979,
   'time_step': 0.0019355219426079057,
   'init_value': -0.7475205659866333,
   'ave_value': -0.7482810573862553,
   'soft_opc': nan}),
 (4,
  {'time_sample_batch': 0.0001283913918671141,
   'time_algorithm_update': 0.0017910381112935059,
   'loss': 0.006516197041351099,
   'time_step': 0.0019766616386813294,
   'in

In [170]:
from d3rlpy.ope import FQE
# metrics to evaluate with
from d3rlpy.metrics.scorer import soft_opc_scorer


ope_dataset = get_dataset([2,4,6,8], path="collected_data/rl_stoch_small.txt") #change if you'd prefer different chunks
ope_train_episodes, ope_test_episodes = train_test_split(ope_dataset, test_size=0.2)

fqe = FQE(algo=model, use_gpu=False) #change this if you have one!
fqe.fit(ope_train_episodes, eval_episodes=ope_test_episodes,
        tensorboard_dir='runs',
        n_epochs=50, n_steps_per_epoch=1000, #change if overfitting/underfitting
        scorers={
           'init_value': initial_state_value_estimation_scorer,
            'ave_value': average_value_estimation_scorer,
           'soft_opc': soft_opc_scorer(return_threshold=0)
        })

[ 0.00000000e+00  7.95731469e+08  1.39310892e-01  1.82000047e-02
 -1.00013420e-04  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
 -7.50230117e-02  3.69851546e-01 -6.00000000e-01]
Read chunk # 3 out of 10000
[ 0.00000000e+00  7.95731469e+08 -1.15389108e-01  1.64000047e-02
 -8.80001342e-03  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
 -3.08831172e-01 -2.48178665e-01  6.00000000e-01]
Read chunk # 5 out of 10000
[ 0.00000000e+00  7.95731469e+08 -1.28589108e-01  1.20000047e-02
  1.99998658e-03  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
  7.75212759e-03 -3.52719043e-01  6.00000000e-01]
Read chunk # 7 out of 10000
[ 0.00000000e+00  7.95731469e+08 -1.03989108e-01  7.00000469e-03
 -8.60001342e-03  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
 -2.65974295e-01 -2.19295880e-01  6.00000000e-01]
Read chunk # 9 out of 10000
torch.Size([111080, 6])
2022-04-06 21:02.19 [debug    ] RoundIterator is selected.
2022-04-06 21:02.19 [info     ] Directory is created at d3rlpy_logs/FQE_2022040621021

Epoch 1/50:   0%|          | 0/878 [00:00<?, ?it/s]

2022-04-06 21:02.21 [info     ] FQE_20220406210219: epoch=1 step=878 epoch=1 metrics={'time_sample_batch': 0.0001383091159724972, 'time_algorithm_update': 0.0019023527045456314, 'loss': 0.0005069225541623121, 'time_step': 0.0021042742327296925, 'init_value': -0.23923806846141815, 'ave_value': -0.23953053405079883, 'soft_opc': nan} step=878


2022-04-06 21:02.21 [info     ] Model parameters are saved to d3rlpy_logs/FQE_20220406210219/model_878.pt


Epoch 2/50:   0%|          | 0/878 [00:00<?, ?it/s]

...

2022-04-06 21:03.56 [info     ] Model parameters are saved to d3rlpy_logs/FQE_20220406210219/model_43022.pt


Epoch 50/50:   0%|          | 0/878 [00:00<?, ?it/s]

2022-04-06 21:03.58 [info     ] FQE_20220406210219: epoch=50 step=43900 epoch=50 metrics={'time_sample_batch': 0.0001237134455548332, 'time_algorithm_update': 0.0016329122838778486, 'loss': 0.03252738003676129, 'time_step': 0.0018108640553467909, 'init_value': -1.825412631034851, 'ave_value': -1.826603330390665, 'soft_opc': nan} step=43900


2022-04-06 21:03.58 [info     ] Model parameters are saved to d3rlpy_logs/FQE_20220406210219/model_43900.pt


[(1,
  {'time_sample_batch': 0.0001383091159724972,
   'time_algorithm_update': 0.0019023527045456314,
   'loss': 0.0005069225541623121,
   'time_step': 0.0021042742327296925,
   'init_value': -0.23923806846141815,
   'ave_value': -0.23953053405079883,
   'soft_opc': nan}),
 (2,
  {'time_sample_batch': 0.00013222862756333753,
   'time_algorithm_update': 0.0017707768768275788,
   'loss': 0.0014712029050831187,
   'time_step': 0.0019621259802294756,
   'init_value': -0.4862314462661743,
   'ave_value': -0.48679262416682534,
   'soft_opc': nan}),
 (3,
  {'time_sample_batch': 0.00012772745858020826,
   'time_algorithm_update': 0.001693446706802264,
   'loss': 0.0034076808783818266,
   'time_step': 0.0018776272315370738,
   'init_value': -0.6859617829322815,
   'ave_value': -0.6864856399976079,
   'soft_opc': nan}),
 (4,
  {'time_sample_batch': 0.00012795311443354927,
   'time_algorithm_update': 0.0017053353216219056,
   'loss': 0.005736621589929113,
   'time_step': 0.0018904692491257652,
 